In [1]:
import pickle
import numpy as np
from dataset import Dataset, load_data, attention_init
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Masking, Dropout, LSTM, Bidirectional, Activation, Conv2D, Conv1D, MaxPool1D, AveragePooling1D, BatchNormalization
from keras.layers.merge import dot
from keras.models import Model, load_model
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras import backend as k

/home/fu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dataset_ger = 'berlin'
dataset_ch = 'CASIA'
max_len = 516
features_number = 34
hidden_unit = 512
dropout_rate = 0.65
lstm_cells = 128
classes = 8
batch = 64
epochs = 5000

In [3]:
DATA_ger, features_ger = load_data(dataset_ger)
DATA_ch, features_ch = load_data(dataset_ch)

In [4]:
mix_label = pickle.load(open('mix_data_labels.p', 'rb'))
mix_feature = pickle.load(open('mix_data_features.p', 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'mix_data_labels.p'

In [ ]:
def test_model(train_data, label_data, data_test1, label_test1, data_test2, label_test2, max_len,features_num, hidden_unit, dp_rate, lstm_cells, classes, epochs, batch_size):
    label_test1 = to_categorical(label_test1, num_classes=classes)
    label_test2 = to_categorical(label_test2, num_classes=classes)
    label_data = to_categorical(label_data, num_classes=classes)
    
    tra_data, val_data, tra_label, val_lable = train_test_split(train_data, label_data, test_size=0.4, random_state=886)
    
#     x_test1, _, y_test1, _ = train_test_split(data_test1, label_test1, test_size=0, random_state=886)
# #     _, x_test_Ch, _, y_test_Ch = train_test_split(data_test, label_test, test_size=1, random_state=886)
#     x_test2, _, y_test2, _ = train_test_split(data_test2, label_test2, test_size=0, random_state=886)
    x_test, val_data, y_label, val_label = train_test_split(val_data, val_lable, test_size=0.5, random_state=886)

    u_train1, u_val1 = attention_init(data_test1.shape[0], label_test1.shape[0], 256, 1.0/256)
    u_train2, u_val2 = attention_init(data_test2.shape[0], label_test2.shape[0], 256, 1.0/256)
    u_test, _ = attention_init(x_test.shape[0], x_test.shape[0], 256, 1.0/256)
    model_lossbest = load_model('./weights_blstm_mix_lossbest2.h5')
    score_3, accuracy_3 = model.evaluate([u_train1,data_test1], label_test1, batch_size=128, verbose=1)
    score_2, accuracy_2 = model.evaluate([u_train2,data_test2], label_test2, batch_size=128, verbose=1)
    score_1, accuracy_1 = model.evaluate([u_test,x_test], y_label, batch_size=128, verbose=1)
#     score_2, accuracy_2 = model.evaluate([u_val2,x_test2], y_test2, batch_size=128, verbose=1)
    print('*******************************************************')
    print("Final test validation accuracy: %s" % accuracy_1)
#     print("Final test2 validation accuracy: %s" % accuracy_2)
    print('*******************************************************')
    print("Final german validation accuracy: %s" % accuracy_3)
    print('*******************************************************')
    print("Final chinese validation accuracy: %s" % accuracy_2)
    print('*******************************************************')

In [ ]:
test_model(train_data=mix_feature, label_data=mix_label, data_test1=features_ger, label_test1=DATA_ger.targets, data_test2=features_ch, label_test2=DATA_ch.targets, max_len=max_len, features_num=features_number, hidden_unit=hidden_unit, dp_rate=dropout_rate,
          lstm_cells=lstm_cells, classes=classes, epochs=epochs, batch_size=batch)

In [3]:
def test_model(val_data, val_label, model_path):
#     val_label = to_categorical(val_label, num_classes=classes)
    
#     x_test, val_data, y_label, val_label = train_test_split(data, label, test_size=0.3, random_state=886)

    u_val, _ = attention_init(val_data.shape[0], val_data.shape[0], 256, 1.0/256)
    model = load_model(model_path)

    score, accuracy = model.evaluate([u_val,val_data], val_label, batch_size=128, verbose=1)

    print('*******************************************************')
    print("Final test validation accuracy: %s" % accuracy)
    print('*******************************************************')

In [4]:
import pickle
ger_data=pickle.load(open('val_data_5cla_ger_features.p','rb'))
ger_label=pickle.load(open('val_label_5cla_ger_labels.p','rb'))
ita_data=pickle.load(open('val_data_5cla_ita_features.p','rb'))
ita_label=pickle.load(open('val_label_5cla_ita_labels.p','rb'))
mix_data=pickle.load(open('val_data_5cla_mix_features.p','rb'))
mix_label=pickle.load(open('val_label_5cla_mix_labels.p','rb'))

In [9]:
from dataset import Dataset, load_data, attention_init
from keras.models import Model, load_model
path_ger='./weights_blstm_mix_val_lossbest_ger_5cal.h5'
path_ita='./weights_blstm_mix_val_lossbest_ita_5cal.h5'
test_model(val_data=ger_data,val_label=ger_label,model_path=path_ger)
test_model(val_data=ita_data,val_label=ita_label,model_path=path_ger)

123/123 [==============================] - 2s 16ms/step
*******************************************************
Final test validation accuracy: 0.7560975551605225
*******************************************************
144/144 [==============================] - 2s 16ms/step
*******************************************************
Final test validation accuracy: 0.2777777777777778
*******************************************************


In [6]:
path_mix='./weights_blstm_mix_5cla_val_lossbest2.h5'
test_model(val_data=mix_data,val_label=mix_label,model_path=path_mix)
test_model(val_data=ita_data,val_label=ita_label,model_path=path_mix)
test_model(val_data=ger_data,val_label=ger_label,model_path=path_mix)

267/267 [==============================] - 2s 7ms/step
*******************************************************
Final test validation accuracy: 0.6966292132599077
*******************************************************
144/144 [==============================] - 2s 11ms/step
*******************************************************
Final test validation accuracy: 0.5972222222222222
*******************************************************
123/123 [==============================] - 1s 11ms/step
*******************************************************
Final test validation accuracy: 0.8130081295967102
*******************************************************


In [10]:
ch_data=pickle.load(open('CASIA_features.p','rb'))
ch_label=to_categorical(pickle.load(open('CASIA_db.p','rb')).targets,5)

In [ ]:
test_model(val_data=ch_data,val_label=ch_label,model_path=path_ger)
test_model(val_data=ch_data,val_label=ch_label,model_path=path_ita)
test_model(val_data=ch_data,val_label=ch_label,model_path=path_mix)

1000/1000 [==============================] - 6s 6ms/step
*******************************************************
Final test validation accuracy: 0.19800000047683716
*******************************************************
